In [1]:
import uuid
import os

from db.DocumentEntry import DocumentEntry
from db.DocumentRepository import DocumentRepository
from datetime import datetime, timedelta

Let's see if the DocumentRepository works by testing the API.

Let's setup docker

In [1]:
# Connect to the database by exec docker compose in your terminal. This exec. a terminal command using Python
os.system("""
docker compose down;
docker compose up -d --build db
""")

NameError: name 'os' is not defined

Let's first provide the data we want to save and load

In [2]:
documentRepository = DocumentRepository()

# Get the current local date and time
now = datetime.now()

entry1 = DocumentEntry("url1", "Title 1", ["heading1"], "page text 1",
                       ["keyword1", "keyword2", "keyword3", "keyword4", "keyword5", "keyword6", "keyword7"], datetime.now(), [], ["myfirstevencoolerexternallink1", "myfirstevencoolerexternallink2"])

entry2 = DocumentEntry("url2", None, ["", None], "",
                        [None], now - timedelta(days=1), [None], [])
# entry2 = DocumentEntry("url2", [], "content2", now - timedelta(days=1))

documentsToSave = [entry1, entry2]

SC: Connected to the db. Now you can go and build the best search engine around!


Let's first clear the db

In [3]:
documentRepository.deleteAllDocuments()
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Deleted all documents.


[]

How does it look after we save 2 elements?

In [4]:
documentRepository.saveAllDocuments(documentsToSave)
allDocuments = documentRepository.loadAllDocuments()
print(allDocuments)
print(len(allDocuments))

SC: All documents saved.
[DocumentEntry[id=8b37b3e9-9..., url=url1, title=Title 1, headings=['heading1'], page_text=page text ..., keywords=['keyword1', 'keyword2', ...], accessed_timestamp=2024-07-01 20:09:46.551647], internal_links=[]], external_links=['myfirstevencoolerexternallink1', 'myfirstevencoolerexternallink2']], DocumentEntry[id=2003d486-5..., url=url2, title=None, headings=['', None], page_text=, keywords=[None], accessed_timestamp=2024-06-30 20:09:46.551566], internal_links=[None]], external_links=[]]]
2


Are the types correct?

In [5]:
print(type(allDocuments))
print(type(allDocuments[0]))

<class 'list'>
<class 'db.DocumentEntry.DocumentEntry'>


The first should be "list", the second "db.DocumentEntry.DocumentEntry".

Let's add one more element

In [6]:
entry3 = DocumentEntry("url3", "Title 3", ["heading3"], "page text 3",
                       ["keyword1"], datetime.now() - timedelta(days=2), ["internalLink"], [None, None], uuid.uuid4())

# entry3 = DocumentEntry("url3", ["keyword5"], "content3", datetime.now() - timedelta(days=2), uuid.uuid4())
documentRepository.saveDocument(entry3)
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Saved document.


[DocumentEntry[id=8b37b3e9-9..., url=url1, title=Title 1, headings=['heading1'], page_text=page text ..., keywords=['keyword1', 'keyword2', ...], accessed_timestamp=2024-07-01 20:09:46.551647], internal_links=[]], external_links=['myfirstevencoolerexternallink1', 'myfirstevencoolerexternallink2']],
 DocumentEntry[id=2003d486-5..., url=url2, title=None, headings=['', None], page_text=, keywords=[None], accessed_timestamp=2024-06-30 20:09:46.551566], internal_links=[None]], external_links=[]],
 DocumentEntry[id=9683da87-5..., url=url3, title=Title 3, headings=['heading3'], page_text=page text ..., keywords=['keyword1'], accessed_timestamp=2024-06-29 20:09:46.578572], internal_links=['internalLink']], external_links=[None, None]]]

Lets update this element

In [7]:
entry3 = DocumentEntry(entry3.url, entry3.title, entry3.headings, entry3.page_text,
                       ["keyword1", "keyword6"], entry3.accessed_timestamp, entry3.internal_links, entry3.external_links, entry3.id)
documentRepository.updateDocument(entry3)
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Updated document.


[DocumentEntry[id=8b37b3e9-9..., url=url1, title=Title 1, headings=['heading1'], page_text=page text ..., keywords=['keyword1', 'keyword2', ...], accessed_timestamp=2024-07-01 20:09:46.551647], internal_links=[]], external_links=['myfirstevencoolerexternallink1', 'myfirstevencoolerexternallink2']],
 DocumentEntry[id=2003d486-5..., url=url2, title=None, headings=['', None], page_text=, keywords=[None], accessed_timestamp=2024-06-30 20:09:46.551566], internal_links=[None]], external_links=[]],
 DocumentEntry[id=9683da87-5..., url=url3, title=Title 3, headings=['heading3'], page_text=page text ..., keywords=['keyword1', 'keyword6'], accessed_timestamp=2024-06-29 20:09:46.578572], internal_links=['internalLink']], external_links=[None, None]]]

Lets delete this elemente again

In [8]:
documentRepository.deleteDocument(entry3)
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Deleted document.


[DocumentEntry[id=8b37b3e9-9..., url=url1, title=Title 1, headings=['heading1'], page_text=page text ..., keywords=['keyword1', 'keyword2', ...], accessed_timestamp=2024-07-01 20:09:46.551647], internal_links=[]], external_links=['myfirstevencoolerexternallink1', 'myfirstevencoolerexternallink2']],
 DocumentEntry[id=2003d486-5..., url=url2, title=None, headings=['', None], page_text=, keywords=[None], accessed_timestamp=2024-06-30 20:09:46.551566], internal_links=[None]], external_links=[]]]

Do we have 0 elements when we decide to clear the index?

In [9]:
documentRepository.deleteAllDocuments()
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Deleted all documents.


[]

When you get "[]", everything was right.

Finally, you should shut down the container and kill it.

In [ ]:
os.system("""
docker compose down;
""")